# Cídigo Auxiliar


In [10]:
import socket as libsock
import random
import struct
import time

addr = "127.0.0.2"
port = 2031


def parse_string_to_wallet(string):
    h, s, L, question_mark = tuple(string.decode().split("@"))
    return int(h), s.encode(), int(L), bool(int(question_mark))


def create_random_string(length):
    allowed_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    random_str = ""
    for i in range(length):
        random_str += allowed_chars[random.randint(0, len(allowed_chars) - 1)]
    return random_str


def create_new_wallet(address):
    h = random.randint(-0x7fff - 1, 0x7fff)  # dos bytes
    s = create_random_string(8).encode()
    L = random.randint(0, 99999999999999)
    question_mark = True if random.random() >= 0.5 else False
    wallet = (h, s, L, question_mark)
    return wallet, struct.pack('h8sL?', *wallet)


def run_server():
    socket = libsock.socket(libsock.AF_INET, libsock.SOCK_DGRAM)  # SOCK_DGRAM es UDP
    print("listening on {}:{}...".format(addr, port))  # logging
    socket.bind((addr, port))  # enlazando al puerto
    while True:  # escucha para siempre
        data, address = socket.recvfrom(1024)  # recibe datos del cliente
        print("Data received from address {}".format(address))  # logging
        print("Data is", data.decode().strip())  # logging
        time.sleep(2) # para que parezca que está procesando
        if "hola" in data.decode():  # chequea que su primer mensaje sea hola
            print("hola received")
            wallet, binary = create_new_wallet(address)  # crea una nueva wallet
            print("new wallet", wallet)
            socket.sendto(binary, address)  # envía el cupón al remitente
            print("sent wallet to address")
            new_data, new_address = socket.recvfrom(1024)  # espera respuesta
            time.sleep(2)  # para que parezca que está procesando
            print(new_data)
            proposed_wallet = parse_string_to_wallet(new_data)  # parsea respuesta
            print("Wallet received", proposed_wallet)
            if proposed_wallet == wallet:  # si son iguales, ganamos
                socket.sendto("¡Felicidades! ganaste 50 KioskitoCoins :)\n".encode(), address)
            else:  # si no, avisar
                socket.sendto("Cupón incorrecto :(\n".encode(), address)
        else:
            socket.sendto("Si no saludas bien no podemos hablar :(\n".encode(), address)
        socket.sendto("Adios!\n".encode(), address)

run_server()

listening on 127.0.0.2:2031...
Data received from address ('127.0.0.1', 39415)
Data is hola
hola received
new wallet (8511, b'RPLXEFIA', 68777575226264, True)
sent wallet to address
b'8511@RPLXEFIA@68777575226264@1'
Wallet received (8511, b'RPLXEFIA', 68777575226264, True)
Data received from address ('127.0.0.1', 33661)
Data is hola
hola received
new wallet (-10443, b'LVEUBZDK', 74763442462353, True)
sent wallet to address
b'-10443@LVEUBZDK@74763442462353@1'
Wallet received (-10443, b'LVEUBZDK', 74763442462353, True)


KeyboardInterrupt: 

In [2]:
import socket as libsock
import random
import struct
import time
class Server:
    def __init__(self,puerto):
        self.port=puerto
        self.anddres="127.0.0.3"
    def run(self):
        socket = libsock.socket(libsock.AF_INET, libsock.SOCK_DGRAM)  # SOCK_DGRAM es UDP
        print("listening on {}:{}...".format(self.anddres, self.port))  # logging
        socket.bind((self.anddres, self.port))  # enlazando al puerto
        data, address = socket.recvfrom(1024)  # recibe datos del cliente
        print("Data received from address {}".format(address))  # logging
       # print("Data is", data.decode().strip())  # logging
        #time.sleep(2) # para que parezca que está procesando
        socket0 = libsock.socket(libsock.AF_INET, libsock.SOCK_DGRAM)
        socket0.connect(("8.8.8.8", 53))
        socket0.send(data)
        data1, address1 = socket0.recvfrom(1024)
        socket.sendto(data1, address)
        
        

In [3]:
Servidor1= Server(2035)
Servidor1.run()

listening on 127.0.0.3:2035...
Data received from address ('127.0.0.1', 44454)


KeyboardInterrupt: 

In [ ]:
import struct
pack('s',)